In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


## Ottengo gli id delle stazioni per ogni città diversa

In [4]:
cities = stations_df['city'].unique()
cities

array(['San Jose', 'Redwood City', 'Mountain View', 'Palo Alto',
       'San Francisco'], dtype=object)

In [5]:
city_stations = {}

for city in cities:
    city_stations[city] = stations_df[stations_df['city'] == city]['id'].unique()

city_stations

{'San Jose': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 80, 84]),
 'Redwood City': array([21, 22, 23, 24, 25, 26, 83]),
 'Mountain View': array([27, 28, 29, 30, 31, 32, 33]),
 'Palo Alto': array([34, 35, 36, 37, 38]),
 'San Francisco': array([41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60,
        61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
        82])}

## Prendo in cosiderazione solo gli id delle stazioni appartenenti alla città di San Francisco

In [6]:
San_Fancisco_stations = city_stations["San Francisco"]
San_Fancisco_stations

array([41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       82])

In [7]:
interval=30
window_width = 5

time_slots = [
    "0-6", 
    "6-10", 
    "10-14", 
    "14-17", 
    "17-20", 
    "20-24"
]

## Training

In [10]:
classifiers = {
    'DecisionTree': tree.DecisionTreeClassifier(random_state=42),   
#     'SVC': svm.SVC(),
#     'LogisticRegression': linear_model.LogisticRegression(random_state=42, solver='liblinear', max_iter=1000),
#     'NaiveBayes': naive_bayes.GaussianNB(),
#     'RandomForest': ensemble.RandomForestClassifier(random_state=42, max_features='sqrt'),
}

Effettuo il finetuning dei modelli tramite la grid search

In [ ]:
decision_tree_parameters = {
    'max_depth': list(range(2, 7)),
    'min_samples_split': list(range(2, 7)),
    'criterion' : ['gini', 'entropy'],
    'class_weight': [None, 'balanced']
}

SVC_parameters = {
    'C': [0.1, 1, 100],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['rbf'],
    'class_weight': [None, 'balanced']
}

logistic_regression_parameters = {
    'penalty': ['l1', 'l2'],
    'C': [1, 0.1, 0.01, 0.001],
    'class_weight': [None, 'balanced']
}

random_forest_parameters = {
    'n_estimators': [10, 100, 1000],
    'criterion' : ['gini', 'entropy'],
    'max_depth': list(range(0, 5)), 
    'min_samples_split': list(range(0, 5)), 
    'class_weight': [None, 'balanced']
}

naive_bayes_parameters = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

classifiers_parameters = {
  'DecisionTree': decision_tree_parameters,
  'SVC': SVC_parameters,
  'LogisticRegression': logistic_regression_parameters,
  'RandomForest': random_forest_parameters,
  'NaiveBayes': naive_bayes_parameters
}

tuned_classifiers = {
    'DecisionTree': [],
    'SVC': [],
    'LogisticRegression': [],
    'RandomForest': [],
    'KNN': []
}


for slot in time_slots:
    print(f"---- TIME SLOT {slot} ----\n")
    for reference_station in San_Fancisco_stations:
        train_df = pd.read_csv(f"./datasets/{interval}_{window_width}/Train_test_with_time/station{reference_station}_train.csv")

        time_slot_train_df = train_df.set_index("time", drop=True)
        time_slot_train_df.index=pd.to_datetime(time_slot_train_df.index)
        start_time= slot.split('-')[0]
        end_time = slot.split('-')[1]
        if end_time != '24':       
            time_slot_train_df = time_slot_train_df.between_time(f'{start_time}:00', f'{end_time}:00')
        else:
            time_slot_train_df = time_slot_train_df.between_time(f'{start_time}:00', '23:59')

        y = time_slot_train_df['status']
        X = time_slot_train_df.drop(columns=['status'])
        X = X.reset_index().drop(columns=['time'])
        
        for scoring in ['accuracy', 'recall', 'precision', 'f1_score']:
            for classifier in classifiers.keys():
                print(f"Classifier: {classifier}" )   

                clf = classifiers[classifier]
                parametrers = classifiers_parameters[classifier]

                #I segunti controlli sono necessari in quanto richiamo, precisione e 
                # f1score hanno necessità di conoscere quale sia la classe positiva (QP)
                if scoring == "recall":
                    scorer = make_scorer(recall_score, pos_label="QP")
                elif scoring == "precision":
                    scorer = make_scorer(precision_score, pos_label="QP")
                elif scoring == "f1_score":
                    scorer = make_scorer(f1_score, pos_label="QP")
                else:
                    scorer = scoring

                #Nel caso di logist regression utilizzo la randomizedSearchCV, in quanto più veloce rispetto alla gridSearchCV
                if classifier == 'LogisticRegression':
                    search_result = RandomizedSearchCV(clf, parametrers, scoring=scorer, random_state=42, n_iter=5, n_jobs=-1)
                else:
                    search_result = GridSearchCV(clf, parametrers, scoring=scorer, cv=3, n_jobs=-1)

                fit_result = search_result.fit(X, y)
                params = fit_result.best_params_

                print("-"*10)
                print(f"Best parameters for classifier {classifier} for the station {reference_station} using the scoring {scoring}: ")
                print(params)
                print()
                print()

                tuned_clf = search_result.best_estimator_

                #salvo il modello dopo il fine tuning
                if (Path.cwd() / f'tuned_models_time_slots/{scoring}/{classifier}/{interval}_{window_width}/{slot}').exists() == False:
                    os.makedirs(f"./tuned_models_time_slots/{scoring}/{classifier}/{interval}_{window_width}/{slot}")
                dump(tuned_clf, f'./tuned_models_time_slots/{scoring}/{classifier}/{interval}_{window_width}/{slot}/station{reference_station}_tuned_model.joblib')


## Testing dei fine tuned models

In [48]:

results_table = {}
thresholds = [0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95]

for slot in time_slots:
    for scoring in ["precision"]:
        result_matrix = []
        for classifier in classifiers.keys():
            for threshold in thresholds:
                file = open(f'test_results/tuned_classifiers_with_threshold_time_slots/results_{slot}_{classifier}_{scoring}_{interval}_{window_width}_({int(threshold*100)}%).txt', "w")
#                 file = open(f'test_results/tuned_classifiers_with_threshold_time_slots(1)/results_{slot}_{classifier}_{scoring}_{interval}_{window_width}_({int(threshold*100)}%).txt', "w")

                file.write(f'TESING RESULTS FOR {classifier} CLASSIFIER WITH THRESHOLD {int(threshold*100)}%:\n\n')
                tot_fp = 0
                tot_tp = 0
                tot_fn = 0
                tot_tn = 0

                for station_id in San_Fancisco_stations:
#                     print (f"------------- {station_id}")
                    # Modelli allenati secondo le diverse fasce orarie
                    model = load(f'./tuned_models_time_slots/{scoring}/{classifier}/{interval}_{window_width}/{slot}/station{station_id}_tuned_model.joblib')
                    
                    # Modelli allenati su tutto il training set
#                     model = load(f'./tuned_models/{scoring}/{classifier}/{interval}_{window_width}/station{station_id}_tuned_model.joblib')
                    
                    test_df = pd.read_csv(f'./datasets/{interval}_{window_width}/Train_test_with_time/station{station_id}_test.csv')
                       
                    time_slot_test_df = test_df.set_index("time", drop=True)
                    time_slot_test_df.index=pd.to_datetime(time_slot_test_df.index)
                    start_time= slot.split('-')[0]
                    end_time = slot.split('-')[1]
                    if end_time != '24':       
                        time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', f'{end_time}:00')
                    else:
                        time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', '23:59')

                    y_test = time_slot_test_df['status']
                    X_test = time_slot_test_df.drop(columns=['status']) 
                    X_test = X_test.reset_index().drop(columns=['time']) 
                    
    #                 pred = model.predict(X_test)
    #                 print (pred)

                    prediction_proba = model.predict_proba(X_test)
                    # prediction_proba è una matrice con due colonne: la prima colonna contiene le probabilità che il sample
                    # appartenga alla classe "N", il secondo contiene la probabilità di appartenenza alla classe "QP".
                    # quindi considerero la seconda colonna.
                    
#                     print (prediction_proba)
                    prediction = []
                    for el in prediction_proba:
                        # nel caso in cui nel test set di una determinata fascia non siano presenti elementi QP
                        # predict proba ritornerà una sola colonna con la probabilità di appartenenza alla classe negativa
                        if len(el)>1:
                            if el[1]>= threshold:
                                prediction.append("QP")
                            else:
                                prediction.append("N")
                        else:
                            if 1-el >= threshold:
                                prediction.append("QP")
                            else:
                                prediction.append("N")

                    cm = confusion_matrix(y_test, prediction, labels=["N", "QP"])

                    str_= f'{classifier} FOR STATION {station_id}' + '\n'
                    str_ += f'Confusion matrix:' + '\n'
                    str_ += str(cm) + '\n'

                    tn, fp, fn, tp = cm.ravel()
                    str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

                    test_accuracy = (tn + tp) / (tn + fp + fn + tp)
        #             test_recall = (tp) / (tp + fn)
        #             test_precision = (tp) / (tp + fp)
                    test_recall = recall_score(y_test, prediction, pos_label='QP', zero_division=0)
                    test_precision = precision_score(y_test, prediction, pos_label='QP', zero_division=0)
                    test_f1_score = f1_score(y_test, prediction, pos_label='QP', zero_division=0)

                    str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; f1_score= {test_f1_score}' +'\n\n'
                    str_+= "-"*10 +'\n\n'

                    tot_fp += fp
                    tot_tp += tp
                    tot_fn += fn
                    tot_tn += tn

                    file.write(str_)
                

                avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
                avg_recall = (tot_tp) / (tot_tp + tot_fn)
                if (tot_tp + tot_fp) != 0.0:
                    avg_precision = (tot_tp) / (tot_tp + tot_fp)
                else:
                    avg_precision = 0.0

                if avg_recall!= 0.0 and avg_precision!= 0.0:
                    avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))
                else:
                    avg_f1_score = 0.0
                result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1_score])
                avg_str = f"AVERAGE VALUES FOR {classifier} TEST:\n"
                avg_str += "Confusion matrix:\n"
                avg_str += f"[[{tot_tn} {tot_fp}]\n[{tot_fn} {tot_tp}]]\n"
                avg_str += f"tot_tp={tot_tp}, tot_fn={tot_fn}, tot_fp={tot_fp}, tot_tn={tot_tn}\n"
                avg_str += f"accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}\n\n"
                
                
                file.write(avg_str)
                file.close()

        result_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_f1_score'], index=pd.Index(thresholds))
        results_table[slot] = result_table
        result_table.to_csv(f"./test_results/tuned_classifiers_with_threshold_time_slots/Overall_results_{slot}_{scoring}_{interval}_{window_width}.csv")

# print(f"[[{all_tn} {all_fp}]\n[{all_fn} {all_tp}]]\n")

#### Calcolo le matrici di confusione globali (sommando quelle diverse fasce orarie) per ciscuna soglia di porbabilità. In questo modo posso considerare le prestazioni medie considerando tutte le fasce orarie.

In [2]:
total_conf_matrix = {}
result_matrix = []
thresholds = [0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
curret_scoring = "precision"
current_interval = 30
current_window_width = 5
current_time_slots = [
    "0-6", 
    "6-10", 
    "10-14", 
    "14-17", 
    "17-20", 
    "20-24"
]

for threshold in thresholds:
    print(f"SOGLIA DI PROBABILITA\':{threshold}")
    all_fp=0
    all_tp=0
    all_fn=0
    all_tn=0
    
    for slot in current_time_slots:
        
        input_file_name = f"results_{slot}_DecisionTree_precision_30_5_({int(threshold*100)}%).txt"
        input_file_path = f"./test_results/tuned_classifiers_with_threshold_time_slots/{input_file_name}" 
        
        with open(input_file_path, "r") as file:
            lines = file.read().splitlines()
            last_line = lines[-3]
#             print (last_line)
            results = last_line.split(', ')
            result_values = []
            for result in results:
#                 print (result)
                value = int(result.split('=')[1])
                result_values.append(value)
        all_tp+=result_values[0]   
        all_fn+=result_values[1]
        all_fp+=result_values[2]
        all_tn+=result_values[3]
        
    avg_accuracy = round((all_tn + all_tp) / (all_tn + all_fp + all_fn + all_tp), 5)
    avg_recall = round((all_tp) / (all_tp + all_fn), 5)
    if (all_tp + all_fp) != 0.0:
        avg_precision = round((all_tp) / (all_tp + all_fp), 5)
    else:
        avg_precision = 0.0

    if avg_recall!= 0.0 and avg_precision!= 0.0:
        avg_f1_score = round(2*(1/((1/avg_recall)+(1/avg_precision))), 5)
    else:
        avg_f1_score = 0.0

#     result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1_score])
    result_matrix.append([avg_recall, avg_precision, avg_f1_score])
    final_str = f"[[{all_tn} {all_fp}]\n[{all_fn} {all_tp}]]\n"
    final_str += f"all_tp={all_tp}, all_fn={all_fn}, all_fp={all_fp}, all_tn={all_tn}\n"
    final_str += f"accuracy={round(avg_accuracy, 5)}; recall={round(avg_recall, 5)}; precision={round(avg_precision, 5)}; f1_score={round(avg_f1_score, 5)}\n\n"
    print (final_str)
    total_conf_matrix[threshold] = [all_tp, all_fn, all_fp, all_tn]

# result_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_f1_score'], index=pd.Index(thresholds))
result_table = pd.DataFrame(result_matrix, columns=['oa_recall', 'oa_precision', 'oa_f1_score'], index=pd.Index(thresholds))
# result_table.to_csv(f"./test_results/tuned_classifiers_with_threshold_time_slots/Overall_results_AllSlots_{curret_scoring}_{current_interval}_{current_window_width}.csv")
# print(result_table.to_latex(bold_rows=True))
result_table

SOGLIA DI PROBABILITA':0.5
[[302223 2674]
[3425 7658]]
all_tp=7658, all_fn=3425, all_fp=2674, all_tn=302223
accuracy=0.9807; recall=0.69097; precision=0.74119; f1_score=0.7152


SOGLIA DI PROBABILITA':0.6
[[302885 2012]
[4141 6942]]
all_tp=6942, all_fn=4141, all_fp=2012, all_tn=302885
accuracy=0.98053; recall=0.62636; precision=0.7753; f1_score=0.69292


SOGLIA DI PROBABILITA':0.7
[[303430 1467]
[5107 5976]]
all_tp=5976, all_fn=5107, all_fp=1467, all_tn=303430
accuracy=0.97919; recall=0.5392; precision=0.8029; f1_score=0.64514


SOGLIA DI PROBABILITA':0.8
[[303849 1048]
[6049 5034]]
all_tp=5034, all_fn=6049, all_fp=1048, all_tn=303849
accuracy=0.97754; recall=0.45421; precision=0.82769; f1_score=0.58654


SOGLIA DI PROBABILITA':0.85
[[304045 852]
[6450 4633]]
all_tp=4633, all_fn=6450, all_fp=852, all_tn=304045
accuracy=0.97689; recall=0.41803; precision=0.84467; f1_score=0.55927


SOGLIA DI PROBABILITA':0.9
[[304109 788]
[6927 4156]]
all_tp=4156, all_fn=6927, all_fp=788, all_tn=304109


,oa_recall,oa_precision,oa_f1_score
0.50,0.69097,0.74119,0.71520
0.60,0.62636,0.77530,0.69292
0.70,0.53920,0.80290,0.64514
0.80,0.45421,0.82769,0.58654
0.85,0.41803,0.84467,0.55927
0.90,0.37499,0.84061,0.51863


In [7]:
#Codice per ottenere il latex delle diverse tabelle
time_slot = "20-24"
res_df = pd.read_csv(f"./test_results/tuned_classifiers_with_threshold_time_slots/Overall_results_{time_slot}_precision_30_5.csv", index_col=0)
res_df = res_df.drop(columns=['avg_accuracy']).drop([0.95])
print(res_df.to_latex(bold_rows=True))
res_df

\begin{tabular}{lrrr}
\toprule
{} &  avg\_recall &  avg\_precision &  avg\_f1\_score \\
\midrule
\textbf{0.50} &    0.814674 &       0.842135 &      0.828177 \\
\textbf{0.60} &    0.791304 &       0.852459 &      0.820744 \\
\textbf{0.70} &    0.789674 &       0.852199 &      0.819746 \\
\textbf{0.80} &    0.760326 &       0.857230 &      0.805876 \\
\textbf{0.85} &    0.715217 &       0.869795 &      0.784969 \\
\textbf{0.90} &    0.536413 &       0.854545 &      0.659098 \\
\bottomrule
\end{tabular}



,avg_recall,avg_precision,avg_f1_score
0.50,0.814674,0.842135,0.828177
0.60,0.791304,0.852459,0.820744
0.70,0.789674,0.852199,0.819746
0.80,0.760326,0.857230,0.805876
0.85,0.715217,0.869795,0.784969
0.90,0.536413,0.854545,0.659098


In [5]:
# RISULTATI MIGLIORI

best_results ={
    "0-6" : [90],     
    "6-10" : [60],    
    "10-14" : [60],     
    "14-17" : [60],     
    "17-20" : [70],     
    "20-24" : [85]   
}

rows = []
result_matrix = []
columns = ["avg_recall", "avg_precision", "avg_f1_score"]

for slot in best_results.keys():
    for prob in best_results[slot]:

        input_file_name = f"results_{slot}_DecisionTree_precision_30_5_({prob}%).txt"
        
        input_file_path = f"./test_results/tuned_classifiers_with_threshold_time_slots/{input_file_name}"

        row = f"{slot}({prob}%)"
        rows.append(row)
        with open(input_file_path, "r") as file:
            lines = file.read().splitlines()
            last_line = lines[-2]
#             print (last_line)
            results = last_line.split(';')
            result_values = []
            for result in results:
                
                value = round(float(result.split('=')[1]), 4)
                result_values.append(value)
            result_matrix.append(result_values[1:])

results_table = pd.DataFrame(result_matrix, columns=columns, index=pd.Index(rows))
print(results_table.to_latex(bold_rows=True))
results_table


\begin{tabular}{lrrr}
\toprule
{} &  avg\_recall &  avg\_precision &  avg\_f1\_score \\
\midrule
\textbf{0-6(90\%)  } &      0.7729 &         0.9376 &        0.8473 \\
\textbf{6-10(60\%) } &      0.3600 &         0.5703 &        0.4414 \\
\textbf{10-14(60\%)} &      0.5398 &         0.6572 &        0.5927 \\
\textbf{14-17(60\%)} &      0.3165 &         0.5498 &        0.4018 \\
\textbf{17-20(70\%)} &      0.2576 &         0.5846 &        0.3576 \\
\textbf{20-24(85\%)} &      0.7152 &         0.8698 &        0.7850 \\
\bottomrule
\end{tabular}



,avg_recall,avg_precision,avg_f1_score
0-6(90%),0.7729,0.9376,0.8473
6-10(60%),0.3600,0.5703,0.4414
10-14(60%),0.5398,0.6572,0.5927
14-17(60%),0.3165,0.5498,0.4018
17-20(70%),0.2576,0.5846,0.3576
20-24(85%),0.7152,0.8698,0.7850


In [ ]:
# plt.figure(figsize=(80,40))
# tree.plot_tree(model, fontsize=10, feature_names=X.columns, class_names=['N', 'QP'], label='all', filled=True, rounded=True)

In [45]:
# DISTRIBUZIONI DEI DATASETS

for slot in time_slots:
    print(f"---- TIME SLOT {slot} ----")
    for reference_station in San_Fancisco_stations:
        train_df = pd.read_csv(f"./datasets/{interval}_{window_width}/Train_test_with_time/station62_train.csv")
        print (f"Number of total training records: {len(train_df)}")
        total_num_QP =  train_df[train_df['status']=='QP']
        print (f"Number of training records QP: {len(total_num_QP)}")
        
        time_slot_train_df = train_df.set_index("time", drop=True)
        time_slot_train_df.index=pd.to_datetime(time_slot_train_df.index)
        start_time= slot.split('-')[0]
        end_time = slot.split('-')[1]
        if end_time != '24':       
            time_slot_train_df = time_slot_train_df.between_time(f'{start_time}:00', f'{end_time}:00')
        else:
            time_slot_train_df = time_slot_train_df.between_time(f'{start_time}:00', '23:59')

        y = time_slot_train_df['status']
        num_QP =  time_slot_train_df[time_slot_train_df['status']=='QP']
        X = time_slot_train_df.drop(columns=['status'])
        X = X.reset_index().drop(columns=['time'])
#         print (f"Number of training records: {len(X)}")
#         print (f"Number of training records QP: {len(num_QP)}")
        
        
        test_df = pd.read_csv(f'./datasets/{interval}_{window_width}/Train_test_with_time/station62_test.csv')
#         print (f"Number of total test records: {len(test_df)}")
#         total_num_QP =  test_df[test_df['status']=='QP']
#         print (f"Number of test records QP: {len(total_num_QP)}") 
        
        time_slot_test_df = test_df.set_index("time", drop=True)
        time_slot_test_df.index=pd.to_datetime(time_slot_test_df.index)
        start_time= slot.split('-')[0]
        end_time = slot.split('-')[1]
        if end_time != '24':       
            time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', f'{end_time}:00')
        else:
            time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', '23:59')

        y_test = time_slot_test_df['status']
        num_QP =  time_slot_test_df[time_slot_test_df['status']=='QP']
        X_test = time_slot_test_df.drop(columns=['status']) 
        X_test = X_test.reset_index().drop(columns=['time']) 
        print (f"Number of test records: {len(X_test)}")
        print (f"Number of test records QP: {len(num_QP)}\n")
        break
    break

---- TIME SLOT 0-6 ----
Number of total training records: 19072
Number of training records QP: 41
Number of test records: 2216
Number of test records QP: 0

